## 캡스톤 프로젝트 - 항공 클럽 
- 우리만의 저렴한 항공권 찾기를 작업한 part 1에 이어 유저의 가입을 받아 가입한 유저에게 할인폭이 큰 항공권들을 이메일로 보내주기

### 1. 1, 2단계 - 고객 등록 코드 만들기 
- 사용자가 고객 등록 프로그램에 접근할 수 있도록 리플잇으로 코드들을 호스팅 하고, 콘솔 링크를 사용자와 공유하기 (우선 금일 여기서 작업은 모두 로컬에서 진행)
    1. 리플잇에서 새 프로젝트 생성 
    2. 우리가 'part 1'에서 작업한 구글 시트에서 새로운 sheet를 생성 
    3. 새로운 sheet에 이름, 성 Email의 3개 열을 추가 
    4. sheety에서 새로운 시트를 동일하게 만들기 
        - 주의: Sheety에 재로그인을 하고, 가격 엔드포인트에 있는 PUT 체크박스에 다시 체크해야 함.
        

### 2. 3단계 - 항공편이 없는 도착지 예외 처리하기 
- 일부 도착지 및 특정 기간에는 이용 가능한 항공편이 없는 경우가 있다. 이러한 상황에 코드가 깨지거나 충돌하지 않도록 예외 처리를 추가해야한다.
    1. 항공권 특가 구글 시트상의 가격표 마지막 행에 Bali, DPS, 520,000을 추가
    2.  프로그램을 실행하면 충돌이 발생할 것이다. max_stopovers를 0으로 설정했지만, 런던에서 덴파사르(발리)로 가는 직항편이 없기 때문이다. 예외 처리를 사용하여 이러한 충돌이 일어나지 않도록 해 보자. 항공편 데이터가 비어 있는 상황을 확인하고 코드가 충돌 없이 계속 실행되도록 하기 위해서는 try/except/else를 사용해야 한다.

In [ ]:
# data_manager.py 

from pprint import pprint
import requests

SHEETY_PRICES_ENDPOINT = YOUR SHEETY PRICES ENDPOINT


class DataManager:

    def __init__(self):
        self.destination_data = {}

    def get_destination_data(self):
        response = requests.get(url=SHEETY_PRICES_ENDPOINT)
        data = response.json()
        self.destination_data = data["prices"]
        return self.destination_data

    def update_destination_codes(self):
        for city in self.destination_data:
            new_data = {
                "price": {
                    "iataCode": city["iataCode"]
                }
            }
            response = requests.put(
                url=f"{SHEETY_PRICES_ENDPOINT}/{city['id']}",
                json=new_data
            )
            print(response.text)

In [ ]:
# flight_search.py

import os
import requests
from flight_data import FlightData
from pprint import pprint
TEQUILA_ENDPOINT = "https://tequila-api.kiwi.com"
TEQUILA_API_KEY = YOUR TEQUILA API KEY

class FlightSearch:

    def __init__(self):
        self.city_codes = []

    def get_destination_codes(self, city_names):
        print("get destination codes triggered")
        location_endpoint = f"{TEQUILA_ENDPOINT}/locations/query"
        headers = {"apikey": TEQUILA_API_KEY}
        for city in city_names:
            query = {"term": city, "location_types": "city"}
            response = requests.get(url=location_endpoint, headers=headers, params=query)
            results = response.json()["locations"]
            code = results[0]["code"]
            self.city_codes.append(code)

        return self.city_codes

    def check_flights(self, origin_city_code, destination_city_code, from_time, to_time):
        print(f"Check flights triggered for {destination_city_code}")
        headers = {"apikey": TEQUILA_API_KEY}
        query = {
            "fly_from": origin_city_code,
            "fly_to": destination_city_code,
            "date_from": from_time.strftime("%d/%m/%Y"),
            "date_to": to_time.strftime("%d/%m/%Y"),
            "nights_in_dst_from": 7,
            "nights_in_dst_to": 30,
            "flight_type": "round",
            "one_for_city": 1,
            "max_stopovers": 0,
            "curr": "GBP"
        }

        response = requests.get(
            url=f"{TEQUILA_ENDPOINT}/v2/search",
            headers=headers,
            params=query,
        )

        ################
        try:
            data = response.json()["data"][0]
            print(f"{destination_city_code} {data['price']}")
        except IndexError:
            print(f"No flights found for {destination_city_code}.")
            return None
        else:
            flight_data = FlightData(
                price=data["price"],
                origin_city=data["route"][0]["cityFrom"],
                origin_airport=data["route"][0]["flyFrom"],
                destination_city=data["route"][0]["cityTo"],
                destination_airport=data["route"][0]["flyTo"],
                out_date=data["route"][0]["local_departure"].split("T")[0],
                return_date=data["route"][1]["local_departure"].split("T")[0]
            )
            return flight_data
        ################

In [ ]:
# main.py

from datetime import datetime, timedelta
from data_manager import DataManager
from flight_search import FlightSearch
from notification_manager import NotificationManager


ORIGIN_CITY_IATA = "ICN"

data_manager = DataManager()
flight_search = FlightSearch()
notification_manager = NotificationManager()

sheet_data = data_manager.get_destination_data()

if sheet_data[0]["iataCode"] == "":
    city_names = [row["city"] for row in sheet_data]
    data_manager.city_codes = flight_search.get_destination_codes(city_names)
    data_manager.update_destination_codes()
    sheet_data = data_manager.get_destination_data()

destinations = {
    data["iataCode"]: {
        "id": data["id"],
        "city": data["city"],
        "price": data["lowestPrice"]
    } for data in sheet_data}

tomorrow = datetime.now() + timedelta(days=1)
six_month_from_today = datetime.now() + timedelta(days=6 * 30)

for destination_code in destinations:
    flight = flight_search.check_flights(
        ORIGIN_CITY_IATA,
        destination_code,
        from_time=tomorrow,
        to_time=six_month_from_today
    )
       
    ################
    if flight is None:
        continue
    ################

    if flight.price < destinations[destination_code]["price"]:
        users = data_manager.get_customer_emails()
        emails = [row["email"] for row in users]
        names = [row["firstName"] for row in users]
        message = f"Low price alert! Only {flight.price}GBP to fly from {flight.origin_city}-{flight.origin_airport} to {flight.destination_city}-{flight.destination_airport}, from {flight.out_date} to {flight.return_date}."
        if flight.stop_overs > 0:
            message += f"\n\nFlight has {flight.stop_overs}, via {flight.via_city}."
        link = f"https://www.google.co.uk/flights?hl=en#flt={flight.origin_airport}.{flight.destination_airport}.{flight.out_date}*{flight.destination_airport}.{flight.origin_airport}.{flight.return_date}"

        notification_manager.send_emails(emails, message, link)

### 3. 4단계 - 직항편이 없는 도착지
- 인기 있는 목적지 중에는 고객들이 가고 싶어 하지만, 직항편이 없는 곳도 많다. 
    1. 항공권이 검색되지 않으면, 1회 경유 하는 항공권이 있는지 확인하여 출력 
    2.  FlightData 클래스를 수정하여, 초깃값을 넣은 2개의 init 파라미터 옵션 stop_overs=0와 via_city=""을 추가. 위 1번에서 얻은 결과를 출력하는 대신, stop_overs 값은 1, via_city는 경유 도시명으로 해서 flight 객체 만들기. 1번에서 출력한 데이터를 주의 깊게 살펴보고, origin_city, origin_airport,
    destination_city, destination_airport, out_date, return_date 정보를 추출.

    힌트: API로 가져온 "route" key 값에는 이제 4가지 요소가 포함된 리스트가 있다.

    [출발지 -> 경유지, 경유지 -> 도착지, 도착지 -> 경유지, 경유지 -> 출발지].

In [ ]:
# data_manager.py
from pprint import pprint
import requests

SHEETY_PRICES_ENDPOINT = YOUR SHEETY PRICES ENDPOINT


class DataManager:

    def __init__(self):
        self.destination_data = {}

    def get_destination_data(self):
        response = requests.get(url=SHEETY_PRICES_ENDPOINT)
        data = response.json()
        self.destination_data = data["prices"]
        return self.destination_data

    def update_destination_codes(self):
        for city in self.destination_data:
            new_data = {
                "price": {
                    "iataCode": city["iataCode"]
                }
            }
            response = requests.put(
                url=f"{SHEETY_PRICES_ENDPOINT}/{city['id']}",
                json=new_data
            )
            print(response.text)

In [ ]:
# flight_data.py 

class FlightData:

    def __init__(self, price, origin_city, origin_airport, destination_city, destination_airport, out_date, return_date, stop_overs=0, via_city=""):
        self.price = price
        self.origin_city = origin_city
        self.origin_airport = origin_airport
        self.destination_city = destination_city
        self.destination_airport = destination_airport
        self.out_date = out_date
        self.return_date = return_date
        
        self.stop_overs = stop_overs
        self.via_city = via_city§

In [ ]:
# flight_search.py

import os
import requests
from flight_data import FlightData
from pprint import pprint
TEQUILA_ENDPOINT = "https://tequila-api.kiwi.com"


class FlightSearch:

    def __init__(self):
        self.city_codes = []

    def get_destination_codes(self, city_names):
        print("get destination codes triggered")
        location_endpoint = f"{TEQUILA_ENDPOINT}/locations/query"
        headers = {"apikey": os.environ["TEQUILA_API_KEY"]}
        for city in city_names:
            query = {"term": city, "location_types": "city"}
            response = requests.get(url=location_endpoint, headers=headers, params=query)
            results = response.json()["locations"]
            code = results[0]["code"]
            self.city_codes.append(code)

        return self.city_codes

    def check_flights(self, origin_city_code, destination_city_code, from_time, to_time):
        print(f"Check flights triggered for {destination_city_code}")
        headers = {"apikey": os.environ["TEQUILA_API_KEY"]}
        query = {
            "fly_from": origin_city_code,
            "fly_to": destination_city_code,
            "date_from": from_time.strftime("%d/%m/%Y"),
            "date_to": to_time.strftime("%d/%m/%Y"),
            "nights_in_dst_from": 7,
            "nights_in_dst_to": 30,
            "flight_type": "round",
            "one_for_city": 1,
            "max_stopovers": 0,
            "curr": "GBP"
        }

        response = requests.get(
            url=f"{TEQUILA_ENDPOINT}/v2/search",
            headers=headers,
            params=query,
        )

        try:
            data = response.json()["data"][0]
        except IndexError:

            ##########################
            query["max_stopovers"] = 1
            response = requests.get(
                url=f"{TEQUILA_ENDPOINT}/v2/search",
                headers=headers,
                params=query,
            )
            data = response.json()["data"][0]
            pprint(data)
            flight_data = FlightData(
                price=data["price"],
                origin_city=data["route"][0]["cityFrom"],
                origin_airport=data["route"][0]["flyFrom"],
                destination_city=data["route"][1]["cityTo"],
                destination_airport=data["route"][1]["flyTo"],
                out_date=data["route"][0]["local_departure"].split("T")[0],
                return_date=data["route"][2]["local_departure"].split("T")[0],
                stop_overs=1,
                via_city=data["route"][0]["cityTo"]
            )
            return flight_data
            ###########################
        else:
            flight_data = FlightData(
                price=data["price"],
                origin_city=data["route"][0]["cityFrom"],
                origin_airport=data["route"][0]["flyFrom"],
                destination_city=data["route"][0]["cityTo"],
                destination_airport=data["route"][0]["flyTo"],
                out_date=data["route"][0]["local_departure"].split("T")[0],
                return_date=data["route"][1]["local_departure"].split("T")[0]
            )

            return flight_data

In [ ]:
# main.py 
from datetime import datetime, timedelta
from data_manager import DataManager
from flight_search import FlightSearch
from notification_manager import NotificationManager


ORIGIN_CITY_IATA = "LON"

data_manager = DataManager()
flight_search = FlightSearch()
notification_manager = NotificationManager()

sheet_data = data_manager.get_destination_data()

if sheet_data[0]["iataCode"] == "":
    city_names = [row["city"] for row in sheet_data]
    data_manager.city_codes = flight_search.get_destination_codes(city_names)
    data_manager.update_destination_codes()
    sheet_data = data_manager.get_destination_data()

destinations = {
    data["iataCode"]: {
        "id": data["id"],
        "city": data["city"],
        "price": data["lowestPrice"]
    } for data in sheet_data}

tomorrow = datetime.now() + timedelta(days=1)
six_month_from_today = datetime.now() + timedelta(days=6 * 30)

for destination_code in destinations:
    flight = flight_search.check_flights(
        ORIGIN_CITY_IATA,
        destination_code,
        from_time=tomorrow,
        to_time=six_month_from_today
    )
    if flight is None:
        continue

    if flight.price < destinations[destination_code]["price"]:
        message = f"Low price alert! Only £{flight.price} to fly from {flight.origin_city}-{flight.origin_airport} to {flight.destination_city}-{flight.destination_airport}, from {flight.out_date} to {flight.return_date}."

        ######################
        if flight.stop_overs > 0:
            message += f"\nFlight has {flight.stop_overs} stop over, via {flight.via_city}."
            print(message)
        #######################

        notification_manager.send_sms(message)

### 4. 5단계 - 전체 고객에게 이메일 보내기 
- 이제 프로그램이 예상대로 잘 동작을 하므로 저렴한 항공권이 뜰 때 고객들에게 알리기만 하면 된다. 
1. NotificationManager 안에 send_emails()라는 메소드를 만등기. smtplib와 이메일 전송에 대해 배운 내용을 바탕으로 구글 시트의 사용자 시트에 있는 고객 모두에게 최저가 항공권 정보가 포함된 이메일을 보내보기.

    - 주의: 이메일을 보낼 때는 ‘£’등의 기호를 넣으면 오류가 날 수 있는데, 메시지를 UTF-8로 인코딩하면 오류를 해결할 수 있다. 

2. 정보가 미리 입력되어 있는 구글 항공권 링크를 생성하여, 사용자가 이메일에 있는 이 링크를 클릭하면 항공권을 예약할 수 있도록 한다.
    - 예) 2020-08-25 과 2020-09-08 사이에 STN에서 SXF로 가는 구글 항공권 링크:

In [ ]:
# data_manager.py

def get_customer_emails(self):
    customers_endpoint = SHEET_USERS_ENDPOINT
    response = requests.get(customers_endpoint)
    data = response.json()
    self.customer_data = data["users"]
    return self.customer_data

In [ ]:
# main.py
from datetime import datetime, timedelta
from data_manager import DataManager
from flight_search import FlightSearch
from notification_manager import NotificationManager


ORIGIN_CITY_IATA = "LON"

data_manager = DataManager()
flight_search = FlightSearch()
notification_manager = NotificationManager()

sheet_data = data_manager.get_destination_data()

if sheet_data[0]["iataCode"] == "":
    city_names = [row["city"] for row in sheet_data]
    data_manager.city_codes = flight_search.get_destination_codes(city_names)
    data_manager.update_destination_codes()
    sheet_data = data_manager.get_destination_data()

destinations = {
    data["iataCode"]: {
        "id": data["id"],
        "city": data["city"],
        "price": data["lowestPrice"]
    } for data in sheet_data}

tomorrow = datetime.now() + timedelta(days=1)
six_month_from_today = datetime.now() + timedelta(days=6 * 30)

for destination_code in destinations:
    flight = flight_search.check_flights(
        ORIGIN_CITY_IATA,
        destination_code,
        from_time=tomorrow,
        to_time=six_month_from_today
    )
    print(flight.price)
    if flight is None:
        continue

    if flight.price < destinations[destination_code]["price"]:

        users = data_manager.get_customer_emails()
        emails = [row["email"] for row in users]
        names = [row["firstName"] for row in users]

        message = f"Low price alert! Only £{flight.price} to fly from {flight.origin_city}-{flight.origin_airport} to {flight.destination_city}-{flight.destination_airport}, from {flight.out_date} to {flight.return_date}."
        if flight.stop_overs > 0:
            message += f"\nFlight has {flight.stop_overs} stop over, via {flight.via_city}."

        notification_manager.send_emails(emails, message)

In [ ]:
# notification_manager.py
def send_emails(self, emails, message):
    with smtplib.SMTP(EMAIL_PROVIDER_SMTP_ADDRESS) as connection:
        connection.starttls()
        connection.login(MY_EMAIL, MY_PASSWORD)
        for email in emails:
            connection.sendmail(
                from_addr=MY_EMAIL,
                to_addrs=email,
                msg=f"Subject:New Low Price Flight!\n\n{message}".encode('utf-8')
            )